<a href="https://colab.research.google.com/github/swarnygora/final-project/blob/main/Project_Code/Group12_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Group12_Code.ipynb
# BME 3053C Final Project: Sleep Stage Classification
# Group 12

import mne
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

# Step 1: Load and preprocess EEG data
def load_eeg_data(file_path):
    """Load EEG data from .edf file using MNE."""
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.filter(0.5, 40)  # Bandpass filter (0.5-40 Hz)
    return raw

def extract_epochs(raw, annotations, epoch_length=30):
    """Extract 30-second epochs and corresponding sleep stage labels."""
    events, event_id = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw, events, event_id, tmin=0, tmax=epoch_length-1, baseline=None)
    labels = [annotations[i] for i in events[:, 2]]  # Map events to sleep stages
    return epochs.get_data(), labels

# Step 2: Prepare dataset
data_files = ['path/to/SC4001E0-PSG.edf']  # Replace with your subset paths
annotations_files = ['path/to/SC4001EC-Hypnogram.edf']
X, y = [], []
for data_file, annot_file in zip(data_files, annotations_files):
    raw = load_eeg_data(data_file)
    annotations = mne.read_annotations(annot_file)
    epochs, labels = extract_epochs(raw, annotations)
    X.append(epochs)
    y.extend(labels)

X = np.concatenate(X, axis=0)  # Shape: (n_epochs, n_channels, n_samples)
y = np.array(y)  # Sleep stage labels (e.g., Wake, N1, N2, N3, REM)

# Step 3: Preprocess data
scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

# Encode labels (e.g., Wake=0, N1=1, N2=2, N3=3, REM=4)
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Build CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 sleep stages
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 6: Evaluate model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Step 7: Visualize results
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy_plot.png')
plt.show()

# Step 8: Save model
model.save('sleep_stage_model.h5')

# Demo: Predict on a sample
sample = X_test[0:1]
prediction = model.predict(sample)
predicted_stage = encoder.inverse_transform([np.argmax(prediction)])
print(f"Predicted Sleep Stage: {predicted_stage[0]}")

ModuleNotFoundError: No module named 'mne'